In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 
from sklearn.utils import shuffle 
from collections import Counter 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import keras
from keras.utils import to_categorical
import os
from keras.layers import *
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from keras.models import Sequential, Model
from keras import optimizers

In [2]:
def parse_arff(filename):
	try:
		file = open(filename)
	except:
		print('file could not found or opened, try with file location')
	columns = []
	data = []

	for line in file:
		if line.startswith('@attribute'):
			line = line.strip()
			temp = line.split()
			columns.append(temp[1])
		elif line.startswith('@') or line.startswith('\n'):
			continue
		else:
			line = line.strip()
			temp = line.split(',')
			data.append(temp)

	return data,columns

In [3]:
data, columns = parse_arff('./data/Phishing_Legitimate_full.arff')
phishing_legitimate_full = pd.DataFrame(data, columns = columns, dtype=np.int8)
data, columns = parse_arff('./data/Training Dataset.arff')
training_dataset = pd.DataFrame(data, columns = columns, dtype=np.int8)
datasetcsv = pd.read_csv('./data/Dataset.csv')
TextFrameImage_features_new = pd.read_csv('./data/TextFrameImage Features_new.csv')
dataset_b = pd.read_csv('./data/dataset_B_05_2020.csv')
ISCXURL2016 = pd.read_csv('./data/Canadian Institute for Cybersecurity/Phishing.csv')
phishingcsv = pd.read_csv('./data/phishing.csv')

phishing_legitimate_full.columns
phishing_legitimate_full.head(5)
training_dataset.columns
training_dataset.head(5)
datasetcsv.columns
datasetcsv.head(5)
TextFrameImage_features_new.columns
TextFrameImage_features_new.head(5)
dataset_b.columns
dataset_b.head(5)
ISCXURL2016.columns
ISCXURL2016.head(5)
phishingcsv.columns
phishingcsv.head(5)

In [4]:
loss_fn = keras.losses.Huber(delta=1.0)
activation = ['softplus', 'softsign', 'selu', 'elu', 'exponential', 'tanh', 'sigmoid', 'relu']
optimizers = ['sgd', 'rmsprop', 'adam', 'adadelta', 'adagrad', 'adamax', 'nadam', 'ftrl']
binary_loss = ['binary_crossentropy', 'hinge', 'squared_hinge', loss_fn]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(phishing_legitimate_full[phishing_legitimate_full.columns[:-1]],phishing_legitimate_full[phishing_legitimate_full.columns[-1]], test_size = 0.3)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [24]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (48,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='softplus'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
219/219 [==============================] - 1s 5ms/step - loss: 0.0911 - accuracy: 0.7479 - val_loss: 0.0692 - val_accuracy: 0.8267
Epoch 2/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0581 - accuracy: 0.8701 - val_loss: 0.0505 - val_accuracy: 0.8887
Epoch 3/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0488 - accuracy: 0.8999 - val_loss: 0.0425 - val_accuracy: 0.9140
Epoch 4/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0437 - accuracy: 0.9093 - val_loss: 0.0418 - val_accuracy: 0.9057
Epoch 5/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0422 - accuracy: 0.9096 - val_loss: 0.0400 - val_accuracy: 0.9123
Epoch 6/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0374 - accuracy: 0.9193 - val_loss: 0.0331 - val_accuracy: 0.9267
Epoch 7/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0342 - accuracy: 0.9259 - val_loss: 0.0321 - val_accuracy: 0.9343
Epoch 

In [25]:
x = x_train.reshape((x_train.shape[0], 1, 48))
xx = x_test.reshape((x_test.shape[0], 1, 48))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,48)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
219/219 [==============================] - 3s 12ms/step - loss: 0.5446 - accuracy: 0.7313 - val_loss: 0.3741 - val_accuracy: 0.8570
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 0.3619 - accuracy: 0.8760 - val_loss: 0.3195 - val_accuracy: 0.8947
Epoch 3/20
219/219 [==============================] - 2s 10ms/step - loss: 0.3257 - accuracy: 0.8951 - val_loss: 0.2619 - val_accuracy: 0.9013
Epoch 4/20
219/219 [==============================] - 2s 8ms/step - loss: 0.2956 - accuracy: 0.9104 - val_loss: 0.2686 - val_accuracy: 0.9123
Epoch 5/20
219/219 [==============================] - 2s 7ms/step - loss: 0.2673 - accuracy: 0.9150 - val_loss: 0.2561 - val_accuracy: 0.9217
Epoch 6/20
219/219 [==============================] - 2s 8ms/step - loss: 0.2629 - accuracy: 0.9156 - val_loss: 0.2723 - val_accuracy: 0.9287
Epoch 7/20
219/219 [==============================] - 2s 9ms/step - loss: 0.2338 - accuracy: 0.9276 - val_loss: 0.2333 - val_accuracy: 0.9293
Epoc

In [26]:
x = x_train.reshape((x_train.shape[0], 4, 4, 3))
xt = x_test.reshape((x_test.shape[0], 4, 4, 3))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(4, 4, 3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
219/219 [==============================] - 2s 10ms/step - loss: 0.7073 - accuracy: 0.6411 - val_loss: 0.5325 - val_accuracy: 0.7547
Epoch 2/20
219/219 [==============================] - 2s 8ms/step - loss: 0.5731 - accuracy: 0.7523 - val_loss: 0.4584 - val_accuracy: 0.8017
Epoch 3/20
219/219 [==============================] - 2s 7ms/step - loss: 0.5568 - accuracy: 0.7750 - val_loss: 0.4598 - val_accuracy: 0.8103
Epoch 4/20
219/219 [==============================] - 2s 8ms/step - loss: 0.4977 - accuracy: 0.7951 - val_loss: 0.4507 - val_accuracy: 0.8323
Epoch 5/20
219/219 [==============================] - 2s 8ms/step - loss: 0.5054 - accuracy: 0.8129 - val_loss: 0.4694 - val_accuracy: 0.8313
Epoch 6/20
219/219 [==============================] - 2s 8ms/step - loss: 0.4719 - accuracy: 0.8090 - val_loss: 0.4678 - val_accuracy: 0.8353
Epoch 7/20
219/219 [==============================] - 2s 8ms/step - loss: 0.4953 - accuracy: 0.8119 - val_loss: 0.4013 - val_accuracy: 0.8520
Epoch

In [157]:
x_train, x_test, y_train, y_test = train_test_split(datasetcsv[datasetcsv.columns[:-1]],datasetcsv[datasetcsv.columns[-1]], test_size = 0.3)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [158]:
x = x_train.reshape((x_train.shape[0], 7, 2, 1))
xt = x_test.reshape((x_test.shape[0], 7, 2, 1))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(7, 2, 1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
241/241 [==============================] - 1s 6ms/step - loss: 0.2412 - accuracy: 0.5634 - val_loss: 0.2216 - val_accuracy: 0.6209
Epoch 2/20
241/241 [==============================] - 1s 5ms/step - loss: 0.2066 - accuracy: 0.6217 - val_loss: 0.1913 - val_accuracy: 0.6512
Epoch 3/20
241/241 [==============================] - 1s 5ms/step - loss: 0.1939 - accuracy: 0.6345 - val_loss: 0.1855 - val_accuracy: 0.6270
Epoch 4/20
241/241 [==============================] - 1s 5ms/step - loss: 0.1876 - accuracy: 0.6431 - val_loss: 0.1835 - val_accuracy: 0.6567
Epoch 5/20
241/241 [==============================] - 1s 5ms/step - loss: 0.1828 - accuracy: 0.6558 - val_loss: 0.1690 - val_accuracy: 0.6736
Epoch 6/20
241/241 [==============================] - 1s 5ms/step - loss: 0.1724 - accuracy: 0.6857 - val_loss: 0.1507 - val_accuracy: 0.6888
Epoch 7/20
241/241 [==============================] - 1s 5ms/step - loss: 0.1581 - accuracy: 0.7086 - val_loss: 0.1143 - val_accuracy: 0.7658
Epoch 

In [159]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (14,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
241/241 [==============================] - 1s 3ms/step - loss: 4.2565 - accuracy: 0.7153 - val_loss: 3.7587 - val_accuracy: 0.7491
Epoch 2/20
241/241 [==============================] - 1s 2ms/step - loss: 3.6565 - accuracy: 0.7569 - val_loss: 3.7699 - val_accuracy: 0.7485
Epoch 3/20
241/241 [==============================] - 1s 2ms/step - loss: 3.6297 - accuracy: 0.7571 - val_loss: 3.7513 - val_accuracy: 0.7512
Epoch 4/20
241/241 [==============================] - 1s 2ms/step - loss: 3.4259 - accuracy: 0.7691 - val_loss: 3.2391 - val_accuracy: 0.7839
Epoch 5/20
241/241 [==============================] - 1s 2ms/step - loss: 2.9249 - accuracy: 0.8019 - val_loss: 3.1800 - val_accuracy: 0.7882
Epoch 6/20
241/241 [==============================] - 1s 2ms/step - loss: 2.2999 - accuracy: 0.8373 - val_loss: 1.6840 - val_accuracy: 0.8785
Epoch 7/20
241/241 [==============================] - 1s 2ms/step - loss: 3.1650 - accuracy: 0.7853 - val_loss: 2.9260 - val_accuracy: 0.8052
Epoch 

In [160]:
x = x_train.reshape((x_train.shape[0], 1, 14))
xx = x_test.reshape((x_test.shape[0], 1, 14))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,14)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
241/241 [==============================] - 2s 7ms/step - loss: 2.5580 - accuracy: 0.8082 - val_loss: 2.4955 - val_accuracy: 0.8221
Epoch 2/20
241/241 [==============================] - 1s 5ms/step - loss: 2.3012 - accuracy: 0.8358 - val_loss: 2.3508 - val_accuracy: 0.8358
Epoch 3/20
241/241 [==============================] - 1s 5ms/step - loss: 1.8793 - accuracy: 0.8577 - val_loss: 1.9946 - val_accuracy: 0.8573
Epoch 4/20
241/241 [==============================] - 2s 6ms/step - loss: 1.7870 - accuracy: 0.8662 - val_loss: 2.0309 - val_accuracy: 0.8579
Epoch 5/20
241/241 [==============================] - 2s 7ms/step - loss: 1.6643 - accuracy: 0.8751 - val_loss: 1.8841 - val_accuracy: 0.8648
Epoch 6/20
241/241 [==============================] - 2s 6ms/step - loss: 1.6179 - accuracy: 0.8805 - val_loss: 1.7873 - val_accuracy: 0.8700
Epoch 7/20
241/241 [==============================] - 2s 6ms/step - loss: 1.8689 - accuracy: 0.8566 - val_loss: 1.8599 - val_accuracy: 0.8673
Epoch 

In [155]:
x_train, x_test, y_train, y_test = train_test_split(TextFrameImage_features_new[TextFrameImage_features_new.columns[:-1]],TextFrameImage_features_new[TextFrameImage_features_new.columns[-1]], test_size = 0.3)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)
y_train = np.where(y_train == 0.1, 0, y_train)
y_test = np.where(y_test == 0.1, 0, y_test)
y_train = np.where(y_train == 0.3, 1, y_train)
y_test = np.where(y_test == 0.3, 1, y_test)

In [156]:
x = x_train.reshape((x_train.shape[0], 7, 5, 1))
xt = x_test.reshape((x_test.shape[0], 7, 5, 1))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(7, 5, 1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
286/286 [==============================] - 2s 7ms/step - loss: 0.1289 - accuracy: 0.5305 - val_loss: 0.1132 - val_accuracy: 0.6132
Epoch 2/20
286/286 [==============================] - 2s 6ms/step - loss: 0.0937 - accuracy: 0.7242 - val_loss: 0.0813 - val_accuracy: 0.7675
Epoch 3/20
286/286 [==============================] - 2s 6ms/step - loss: 0.0828 - accuracy: 0.7604 - val_loss: 0.0793 - val_accuracy: 0.7657
Epoch 4/20
286/286 [==============================] - 2s 7ms/step - loss: 0.0818 - accuracy: 0.7641 - val_loss: 0.0811 - val_accuracy: 0.7483
Epoch 5/20
286/286 [==============================] - 2s 8ms/step - loss: 0.0807 - accuracy: 0.7668 - val_loss: 0.0779 - val_accuracy: 0.7759
Epoch 6/20
286/286 [==============================] - 2s 7ms/step - loss: 0.0800 - accuracy: 0.7683 - val_loss: 0.0794 - val_accuracy: 0.7649
Epoch 7/20
286/286 [==============================] - 2s 6ms/step - loss: 0.0790 - accuracy: 0.7740 - val_loss: 0.0769 - val_accuracy: 0.7710
Epoch 

In [63]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (35,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='softplus'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
286/286 [==============================] - 1s 3ms/step - loss: 0.0530 - accuracy: 0.8613 - val_loss: 0.0324 - val_accuracy: 0.9156
Epoch 2/20
286/286 [==============================] - 1s 2ms/step - loss: 0.0275 - accuracy: 0.9284 - val_loss: 0.0282 - val_accuracy: 0.9255
Epoch 3/20
286/286 [==============================] - 1s 2ms/step - loss: 0.0242 - accuracy: 0.9381 - val_loss: 0.0328 - val_accuracy: 0.9131
Epoch 4/20
286/286 [==============================] - 1s 2ms/step - loss: 0.0224 - accuracy: 0.9417 - val_loss: 0.0228 - val_accuracy: 0.9386
Epoch 5/20
286/286 [==============================] - 1s 2ms/step - loss: 0.0217 - accuracy: 0.9437 - val_loss: 0.0221 - val_accuracy: 0.9386
Epoch 6/20
286/286 [==============================] - 1s 2ms/step - loss: 0.0203 - accuracy: 0.9472 - val_loss: 0.0215 - val_accuracy: 0.9431
Epoch 7/20
286/286 [==============================] - 1s 2ms/step - loss: 0.0193 - accuracy: 0.9503 - val_loss: 0.0196 - val_accuracy: 0.9452
Epoch 

In [65]:
x = x_train.reshape((x_train.shape[0], 1, 35))
xx = x_test.reshape((x_test.shape[0], 1, 35))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,35)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
286/286 [==============================] - 2s 7ms/step - loss: 0.5363 - accuracy: 0.7188 - val_loss: 0.2241 - val_accuracy: 0.9192
Epoch 2/20
286/286 [==============================] - 2s 5ms/step - loss: 0.2019 - accuracy: 0.9188 - val_loss: 0.1889 - val_accuracy: 0.9207
Epoch 3/20
286/286 [==============================] - 2s 5ms/step - loss: 0.1875 - accuracy: 0.9241 - val_loss: 0.1833 - val_accuracy: 0.9233
Epoch 4/20
286/286 [==============================] - 2s 6ms/step - loss: 0.1858 - accuracy: 0.9275 - val_loss: 0.1781 - val_accuracy: 0.9227
Epoch 5/20
286/286 [==============================] - 2s 7ms/step - loss: 0.1784 - accuracy: 0.9281 - val_loss: 0.1741 - val_accuracy: 0.9233
Epoch 6/20
286/286 [==============================] - 2s 5ms/step - loss: 0.1730 - accuracy: 0.9298 - val_loss: 0.1700 - val_accuracy: 0.9235
Epoch 7/20
286/286 [==============================] - 2s 5ms/step - loss: 0.1738 - accuracy: 0.9309 - val_loss: 0.1661 - val_accuracy: 0.9258
Epoch 

In [ ]:
phishing_legitimate_full, training_dataset, datasetcsv, TextFrameImage_features_new, dataset_b, ISCXURL2016, phishingcsv

In [150]:
x_train, x_test, y_train, y_test = train_test_split(dataset_b[dataset_b.columns[1:-1]], dataset_b[dataset_b.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == 'phishing', 0, y_train)
y_test = np.where(y_test == 'phishing', 0, y_test)
y_train = np.where(y_train == 'legitimate', 1, y_train)
y_test = np.where(y_test == 'legitimate', 1, y_test)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [151]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (87,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
251/251 [==============================] - 1s 3ms/step - loss: 6.6449 - accuracy: 0.5536 - val_loss: 5.9267 - val_accuracy: 0.5978
Epoch 2/20
251/251 [==============================] - 1s 2ms/step - loss: 5.7425 - accuracy: 0.6098 - val_loss: 5.9502 - val_accuracy: 0.5987
Epoch 3/20
251/251 [==============================] - 1s 2ms/step - loss: 5.1157 - accuracy: 0.6534 - val_loss: 5.3336 - val_accuracy: 0.6366
Epoch 4/20
251/251 [==============================] - 1s 2ms/step - loss: 5.0425 - accuracy: 0.6603 - val_loss: 5.8943 - val_accuracy: 0.6022
Epoch 5/20
251/251 [==============================] - 1s 2ms/step - loss: 5.1700 - accuracy: 0.6532 - val_loss: 4.7736 - val_accuracy: 0.6812
Epoch 6/20
251/251 [==============================] - 1s 2ms/step - loss: 4.6438 - accuracy: 0.6873 - val_loss: 4.3854 - val_accuracy: 0.7014
Epoch 7/20
251/251 [==============================] - 1s 2ms/step - loss: 4.7101 - accuracy: 0.6828 - val_loss: 4.7214 - val_accuracy: 0.6833
Epoch 

In [82]:
x = x_train.reshape((x_train.shape[0], 1, 87))
xx = x_test.reshape((x_test.shape[0], 1, 87))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,87)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
251/251 [==============================] - 2s 8ms/step - loss: 5.4114 - accuracy: 0.5943 - val_loss: 5.0304 - val_accuracy: 0.6416
Epoch 2/20
251/251 [==============================] - 1s 6ms/step - loss: 4.0293 - accuracy: 0.6868 - val_loss: 4.1438 - val_accuracy: 0.6897
Epoch 3/20
251/251 [==============================] - 2s 6ms/step - loss: 3.5602 - accuracy: 0.7134 - val_loss: 3.1274 - val_accuracy: 0.7507
Epoch 4/20
251/251 [==============================] - 1s 5ms/step - loss: 3.3996 - accuracy: 0.7244 - val_loss: 3.1229 - val_accuracy: 0.7381
Epoch 5/20
251/251 [==============================] - 1s 5ms/step - loss: 3.3707 - accuracy: 0.7257 - val_loss: 3.1803 - val_accuracy: 0.7448
Epoch 6/20
251/251 [==============================] - 1s 5ms/step - loss: 3.2807 - accuracy: 0.7337 - val_loss: 3.1578 - val_accuracy: 0.7474
Epoch 7/20
251/251 [==============================] - 1s 6ms/step - loss: 3.2263 - accuracy: 0.7454 - val_loss: 3.0404 - val_accuracy: 0.7684
Epoch 

In [148]:
x = x_train.reshape((x_train.shape[0], 29, 3, 1))
xt = x_test.reshape((x_test.shape[0], 29, 3, 1))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(29, 3, 1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
251/251 [==============================] - 3s 11ms/step - loss: 0.2511 - accuracy: 0.4992 - val_loss: 0.2326 - val_accuracy: 0.5322
Epoch 2/20
251/251 [==============================] - 2s 9ms/step - loss: 0.2428 - accuracy: 0.5123 - val_loss: 0.2281 - val_accuracy: 0.5281
Epoch 3/20
251/251 [==============================] - 2s 10ms/step - loss: 0.2369 - accuracy: 0.5248 - val_loss: 0.2198 - val_accuracy: 0.5427
Epoch 4/20
251/251 [==============================] - 3s 10ms/step - loss: 0.2295 - accuracy: 0.5336 - val_loss: 0.2069 - val_accuracy: 0.5646
Epoch 5/20
251/251 [==============================] - 2s 9ms/step - loss: 0.2230 - accuracy: 0.5436 - val_loss: 0.2014 - val_accuracy: 0.5707
Epoch 6/20
251/251 [==============================] - 2s 8ms/step - loss: 0.2152 - accuracy: 0.5626 - val_loss: 0.1871 - val_accuracy: 0.6010
Epoch 7/20
251/251 [==============================] - 2s 8ms/step - loss: 0.2036 - accuracy: 0.5816 - val_loss: 0.1736 - val_accuracy: 0.6331
Epo

In [165]:
x_train, x_test, y_train, y_test = train_test_split(ISCXURL2016[ISCXURL2016.columns[:-1]],ISCXURL2016[ISCXURL2016.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == 'phishing', 0, y_train)
y_test = np.where(y_test == 'phishing', 0, y_test)
y_train = np.where(y_train == 'benign', 1, y_train)
y_test = np.where(y_test == 'benign', 1, y_test)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [166]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (79,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='tanh'))
 
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
337/337 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.4926 - val_loss: nan - val_accuracy: 0.4951
Epoch 2/20
337/337 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.4930 - val_loss: nan - val_accuracy: 0.4951
Epoch 3/20
337/337 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.4930 - val_loss: nan - val_accuracy: 0.4951
Epoch 4/20
337/337 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.4930 - val_loss: nan - val_accuracy: 0.4951
Epoch 5/20
337/337 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.4930 - val_loss: nan - val_accuracy: 0.4951
Epoch 6/20
337/337 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.4930 - val_loss: nan - val_accuracy: 0.4951
Epoch 7/20
337/337 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.4930 - val_loss: nan - val_accuracy: 0.4951
Epoch 8/20
337/337 [============================

In [146]:
x = x_train.reshape((x_train.shape[0], 1, 79))
xx = x_test.reshape((x_test.shape[0], 1, 79))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,79)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
337/337 [==============================] - 2s 7ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 2/20
337/337 [==============================] - 2s 6ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 3/20
337/337 [==============================] - 2s 7ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 4/20
337/337 [==============================] - 2s 7ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 5/20
337/337 [==============================] - 2s 6ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 6/20
337/337 [==============================] - 2s 7ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 7/20
337/337 [==============================] - 2s 5ms/step - loss: nan - accuracy: 0.4935 - val_loss: nan - val_accuracy: 0.4940
Epoch 8/20
337/337 [============================

In [142]:
x = x_train.reshape((x_train.shape[0], 79, 1, 1))
xt = x_test.reshape((x_test.shape[0], 79, 1, 1))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(79, 1, 1)))
model.add(keras.layers.MaxPool2D(1,1))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
337/337 [==============================] - 13s 38ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 2/20
337/337 [==============================] - 13s 37ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 3/20
337/337 [==============================] - 15s 44ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 4/20
337/337 [==============================] - 15s 44ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 5/20
337/337 [==============================] - 15s 45ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 6/20
337/337 [==============================] - 12s 36ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 7/20
337/337 [==============================] - 13s 38ms/step - loss: nan - accuracy: 0.4949 - val_loss: nan - val_accuracy: 0.4908
Epoch 8/20
337/337 [==============

In [133]:
x_train, x_test, y_train, y_test = train_test_split(phishingcsv[phishingcsv.columns[:-1]],phishingcsv[phishingcsv.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [138]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (31,),  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
242/242 [==============================] - 1s 4ms/step - loss: 0.2228 - accuracy: 0.5608 - val_loss: 0.2271 - val_accuracy: 0.5457
Epoch 2/20
242/242 [==============================] - 1s 3ms/step - loss: 0.2188 - accuracy: 0.5622 - val_loss: 0.2271 - val_accuracy: 0.5457
Epoch 3/20
242/242 [==============================] - 1s 2ms/step - loss: 0.2187 - accuracy: 0.5625 - val_loss: 0.2270 - val_accuracy: 0.5457
Epoch 4/20
242/242 [==============================] - 1s 2ms/step - loss: 0.2187 - accuracy: 0.5625 - val_loss: 0.2269 - val_accuracy: 0.5463
Epoch 5/20
242/242 [==============================] - 1s 2ms/step - loss: 0.3694 - accuracy: 0.5381 - val_loss: 1.0442 - val_accuracy: 0.4546
Epoch 6/20
242/242 [==============================] - 1s 2ms/step - loss: 0.5028 - accuracy: 0.5160 - val_loss: 0.2269 - val_accuracy: 0.5457
Epoch 7/20
242/242 [==============================] - 1s 2ms/step - loss: 0.2186 - accuracy: 0.5628 - val_loss: 0.2269 - val_accuracy: 0.5460
Epoch 

In [136]:
x = x_train.reshape((x_train.shape[0], 1, 31))
xx = x_test.reshape((x_test.shape[0], 1, 31))

model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,31)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
242/242 [==============================] - 2s 9ms/step - loss: 0.2589 - accuracy: 0.4561 - val_loss: 0.2154 - val_accuracy: 0.4941
Epoch 2/20
242/242 [==============================] - 2s 7ms/step - loss: 0.2501 - accuracy: 0.4635 - val_loss: 0.2126 - val_accuracy: 0.5104
Epoch 3/20
242/242 [==============================] - 2s 8ms/step - loss: 0.1724 - accuracy: 0.5656 - val_loss: 0.0699 - val_accuracy: 0.8092
Epoch 4/20
242/242 [==============================] - 2s 10ms/step - loss: 0.0980 - accuracy: 0.7111 - val_loss: 0.1039 - val_accuracy: 0.6262
Epoch 5/20
242/242 [==============================] - 2s 8ms/step - loss: 0.0879 - accuracy: 0.7449 - val_loss: 0.0423 - val_accuracy: 0.8939
Epoch 6/20
242/242 [==============================] - 1s 6ms/step - loss: 0.0584 - accuracy: 0.8453 - val_loss: 0.1060 - val_accuracy: 0.6216
Epoch 7/20
242/242 [==============================] - 1s 5ms/step - loss: 0.0697 - accuracy: 0.8047 - val_loss: 0.0512 - val_accuracy: 0.8423
Epoch

In [135]:
x = x_train.reshape((x_train.shape[0], 31, 1, 1))
xt = x_test.reshape((x_test.shape[0], 31, 1, 1))

model = keras.Sequential()
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(31, 1, 1)))
model.add(keras.layers.MaxPool2D(1,1))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='tanh'))

model.compile(loss=loss_fn, optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

Epoch 1/20
242/242 [==============================] - 4s 18ms/step - loss: 0.2205 - accuracy: 0.5611 - val_loss: 0.2267 - val_accuracy: 0.5463
Epoch 2/20
242/242 [==============================] - 4s 17ms/step - loss: 0.2187 - accuracy: 0.5622 - val_loss: 0.2260 - val_accuracy: 0.5460 
Epoch 3/20
242/242 [==============================] - 4s 16ms/step - loss: 0.2174 - accuracy: 0.5656 - val_loss: 0.2256 - val_accuracy: 0.5475
Epoch 4/20
242/242 [==============================] - 4s 16ms/step - loss: 0.2170 - accuracy: 0.5659 - val_loss: 0.2248 - val_accuracy: 0.5493
Epoch 5/20
242/242 [==============================] - 4s 16ms/step - loss: 0.2160 - accuracy: 0.5683 - val_loss: 0.2236 - val_accuracy: 0.5496
Epoch 6/20
242/242 [==============================] - 4s 16ms/step - loss: 0.2156 - accuracy: 0.5706 - val_loss: 0.2231 - val_accuracy: 0.5532
Epoch 7/20
242/242 [==============================] - 4s 16ms/step - loss: 0.2151 - accuracy: 0.5739 - val_loss: 0.2207 - val_accuracy: 0.558

In [ ]:
phishing_legitimate_full, training_dataset, datasetcsv, TextFrameImage_features_new, dataset_b, ISCXURL2016, phishingcsv

In [118]:
len(phishing_legitimate_full), len(training_dataset), len(datasetcsv), len(TextFrameImage_features_new), len(dataset_b), len(ISCXURL2016), len(phishingcsv)

(10000, 11055, 11000, 13071, 11430, 15367, 11054)

In [112]:
phishing_legitimate_full.columns, datasetcsv.columns

(Index(['NumDots', 'SubdomainLevel', 'PathLevel', 'UrlLength', 'NumDash',
        'NumDashInHostname', 'AtSymbol', 'TildeSymbol', 'NumUnderscore',
        'NumPercent', 'NumQueryComponents', 'NumAmpersand', 'NumHash',
        'NumNumericChars', 'NoHttps', 'RandomString', 'IpAddress',
        'DomainInSubdomains', 'DomainInPaths', 'HttpsInHostname',
        'HostnameLength', 'PathLength', 'QueryLength', 'DoubleSlashInPath',
        'NumSensitiveWords', 'EmbeddedBrandName', 'PctExtHyperlinks',
        'PctExtResourceUrls', 'ExtFavicon', 'InsecureForms',
        'RelativeFormAction', 'ExtFormAction', 'AbnormalFormAction',
        'PctNullSelfRedirectHyperlinks', 'FrequentDomainNameMismatch',
        'FakeLinkInStatusBar', 'RightClickDisabled', 'PopUpWindow',
        'SubmitInfoToEmail', 'IframeOrFrame', 'MissingTitle',
        'ImagesOnlyInForm', 'SubdomainLevelRT', 'UrlLengthRT',
        'PctExtResourceUrlsRT', 'AbnormalExtFormActionR', 'ExtMetaScriptLinkRT',
        'PctExtNullSelfRedir

In [117]:
ISCXURL2016.columns, dataset_b.columns

(Index(['Querylength', 'domain_token_count', 'path_token_count',
        'avgdomaintokenlen', 'longdomaintokenlen', 'avgpathtokenlen', 'tld',
        'charcompvowels', 'charcompace', 'ldl_url', 'ldl_domain', 'ldl_path',
        'ldl_filename', 'ldl_getArg', 'dld_url', 'dld_domain', 'dld_path',
        'dld_filename', 'dld_getArg', 'urlLen', 'domainlength', 'pathLength',
        'subDirLen', 'fileNameLen', 'this.fileExtLen', 'ArgLen', 'pathurlRatio',
        'ArgUrlRatio', 'argDomanRatio', 'domainUrlRatio', 'pathDomainRatio',
        'argPathRatio', 'executable', 'isPortEighty', 'NumberofDotsinURL',
        'ISIpAddressInDomainName', 'CharacterContinuityRate',
        'LongestVariableValue', 'URL_DigitCount', 'host_DigitCount',
        'Directory_DigitCount', 'File_name_DigitCount', 'Extension_DigitCount',
        'Query_DigitCount', 'URL_Letter_Count', 'host_letter_count',
        'Directory_LetterCount', 'Filename_LetterCount',
        'Extension_LetterCount', 'Query_LetterCount', 'Lo

In [116]:
TextFrameImage_features_new.columns, phishingcsv.columns

(Index(['has_ip', 'long_url', 'short_service', 'has_at',
        'double_slash_redirect', 'pref_suf', 'has_sub_domain', 'ssl_state',
        'long_domain', 'port', 'https_token', 'req_url', 'url_of_anchor',
        'submit_to_email', 'domain_Age', 'dns_record', 'traffic', 'page_rank',
        'links_to_page', 'links_to_page.1', 'right_click', 'popup', 'redirect',
        'tag_links', 'shf', 'sty_similarity', 'lay_similarity', 'stats_report',
        'favicon', 'Image_size', 'alt_text', 'form_login', 'mouseover',
        'iframe_redirection', 'has_https', 'target'],
       dtype='object'),
 Index(['Index', 'UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
        'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
        'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
        'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
        'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
        'UsingPopupWindow', 'IframeRed

In [173]:
x_train, x_test, y_train, y_test = train_test_split(training_dataset[training_dataset.columns[:-1]],training_dataset[training_dataset.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)
x_train = np.array(x_train).astype(np.int16)
y_train = np.array(y_train).astype(np.int16)
x_test = np.array(x_test).astype(np.int16)
y_test = np.array(y_test).astype(np.int16)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(phishingcsv[phishingcsv.columns[:-1]],phishingcsv[phishingcsv.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(dataset_b[dataset_b.columns[1:-1]], dataset_b[dataset_b.columns[-1]], test_size = 0.3)
y_train = np.where(y_train == 'phishing', 0, y_train)
y_test = np.where(y_test == 'phishing', 0, y_test)
y_train = np.where(y_train == 'legitimate', 1, y_train)
y_test = np.where(y_test == 'legitimate', 1, y_test)
x_train = np.array(x_train).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
x_test = np.array(x_test).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(32, activation='relu',return_sequences=True, input_shape=(29, 3)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(keras.layers.Reshape((32, 2, 2)))
model.add(keras.layers.Conv2D(128,(1,1), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(1,activation='tanh'))

x = x_train.reshape((x_train.shape[0], 29, 3))
xt = x_test.reshape((x_test.shape[0], 29, 3))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xt, y_test))

In [ ]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (31,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(keras.layers.Reshape((32, 4)))
model.add(keras.layers.LSTM(32, activation='relu',return_sequences=True, input_shape=(32, 4)))
model.add(Flatten())
model.add(keras.layers.Reshape((32, 16, 2)))
model.add(keras.layers.Conv2D(128, (1,1), input_shape=(32, 15, 2), activation='relu'))
model.add(Flatten())
model.add(Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

In [ ]:
model = keras.Sequential()
model.add(Dense(64, input_shape = (31,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(keras.layers.Reshape((16, 4, 2)))
model.add(keras.layers.Conv2D(128, (1,1), input_shape=(16, 4, 2), activation='relu'))
model.add(Flatten())
model.add(keras.layers.Reshape((64, 128)))
model.add(keras.layers.LSTM(32, activation='relu',return_sequences=True, input_shape=(64, 128)))
model.add(Flatten())
model.add(Dense(1,activation='softplus'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

In [13]:
model = keras.Sequential()
model.add(keras.layers.LSTM(64, activation='relu',return_sequences=True, input_shape=(1,31)))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64 ,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))
model.add(keras.layers.Dense(30,activation='relu'))
model.add(keras.layers.Reshape((2,5,3)))
model.add(keras.layers.Conv2D(64,(1,1), activation='relu',input_shape=(2,5,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Conv2D(128,(1,1),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1,activation='softplus'))

x = x_train.reshape((x_train.shape[0], 1, 31))
xx = x_test.reshape((x_test.shape[0], 1, 31))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
modelhistory = model.fit(x, y_train, epochs = 20, validation_data=(xx, y_test))

Epoch 1/20
242/242 [==============================] - 3s 11ms/step - loss: 0.6890 - accuracy: 0.5604 - val_loss: 0.6887 - val_accuracy: 0.5499
Epoch 2/20
242/242 [==============================] - 2s 8ms/step - loss: 0.6871 - accuracy: 0.5591 - val_loss: 0.6881 - val_accuracy: 0.5499
Epoch 3/20
242/242 [==============================] - 2s 8ms/step - loss: 0.6866 - accuracy: 0.5599 - val_loss: 0.6880 - val_accuracy: 0.5499
Epoch 4/20
242/242 [==============================] - 2s 8ms/step - loss: 0.6867 - accuracy: 0.5595 - val_loss: 0.6866 - val_accuracy: 0.5598
Epoch 5/20
242/242 [==============================] - 2s 8ms/step - loss: 0.6673 - accuracy: 0.6076 - val_loss: 0.6278 - val_accuracy: 0.6385
Epoch 6/20
242/242 [==============================] - 2s 8ms/step - loss: 0.6888 - accuracy: 0.5642 - val_loss: 0.6863 - val_accuracy: 0.5617
Epoch 7/20
242/242 [==============================] - 2s 8ms/step - loss: 0.6629 - accuracy: 0.6015 - val_loss: 0.6466 - val_accuracy: 0.6114
Epoch